# Structured Outputs with Pydantic

Learn how to get validated, type-safe responses from LLMs using Pydantic models.

**What You'll Learn:**
- Using `response_model` for JSON Schema validation
- How lionpride renders schemas to prompts
- Handling validation failures and retries
- Complex nested models
- Combining structured outputs with reasoning

**Prerequisites:** Basic understanding of Session, Branch, and communicate()

In [ ]:
import json
from typing import Literal

from pydantic import BaseModel, Field

from lionpride import Session
from lionpride.libs.schema_handlers import typescript_schema
from lionpride.operations import communicate, operate
from lionpride.services import iModel

## 1. Basic Structured Output

Pass a Pydantic model as `response_model` to get validated JSON output.

In [ ]:
# Define a Pydantic model for the expected output
class Sentiment(BaseModel):
    """Sentiment analysis result."""

    text: str = Field(..., description="The analyzed text")
    sentiment: Literal["positive", "negative", "neutral"] = Field(
        ..., description="Overall sentiment"
    )
    confidence: float = Field(..., ge=0.0, le=1.0, description="Confidence score (0-1)")
    key_phrases: list[str] = Field(
        default_factory=list, description="Key phrases that influenced the sentiment"
    )


print("Pydantic Model Schema:")
print(json.dumps(Sentiment.model_json_schema(), indent=2))

In [ ]:
async def analyze_sentiment():
    """Get structured sentiment analysis."""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0)
    session.services.register(model)
    branch = session.create_branch(name="sentiment")

    result = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Analyze the sentiment of this review",
            "context": {
                "review": "The product exceeded my expectations! Fast shipping, "
                "great quality, and the customer service was incredibly helpful."
            },
            "imodel": model.name,
            "response_model": Sentiment,  # Request structured output
            "return_as": "model",  # Return as Pydantic model instance
        },
    )

    print(f"Type: {type(result).__name__}")
    print(f"Sentiment: {result.sentiment}")
    print(f"Confidence: {result.confidence}")
    print(f"Key phrases: {result.key_phrases}")

    return result


result = await analyze_sentiment()

## 2. How lionpride Renders Schemas

lionpride converts JSON Schema to compact TypeScript notation for efficiency.

In [ ]:
# Compare JSON Schema vs TypeScript representation
json_schema = Sentiment.model_json_schema()
ts_schema = typescript_schema(json_schema)

print("=== JSON Schema ===")
json_str = json.dumps(json_schema, indent=2)
print(json_str)
print(f"\nLength: {len(json_str)} chars, ~{len(json_str) // 4} tokens")

print("\n=== TypeScript Schema ===")
print(ts_schema)
print(f"\nLength: {len(ts_schema)} chars, ~{len(ts_schema) // 4} tokens")

reduction = (1 - len(ts_schema) / len(json_str)) * 100
print(f"\nReduction: ~{reduction:.0f}%")

## 3. Complex Nested Models

Structured outputs work with arbitrarily complex nested Pydantic models.

In [ ]:
# Define nested models
class Address(BaseModel):
    street: str
    city: str
    country: str
    postal_code: str


class Employment(BaseModel):
    company: str
    position: str
    start_year: int
    end_year: int | None = None
    is_current: bool = False


class Person(BaseModel):
    """Extracted person information."""

    name: str = Field(..., description="Full name")
    age: int | None = Field(None, description="Age if mentioned")
    occupation: str | None = Field(None, description="Current occupation")
    address: Address | None = Field(None, description="Address if mentioned")
    employment_history: list[Employment] = Field(default_factory=list, description="Work history")
    skills: list[str] = Field(default_factory=list, description="Skills/expertise")


# Show the TypeScript schema
print("TypeScript Schema for Person:")
print(typescript_schema(Person.model_json_schema()))

In [ ]:
async def extract_person_info():
    """Extract structured person information from text."""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o", temperature=0)
    session.services.register(model)
    branch = session.create_branch(name="extraction")

    bio = """
    Sarah Chen is a 32-year-old software architect based in San Francisco. 
    She currently works at TechCorp as VP of Engineering, a position she's 
    held since 2022. Before that, she was a senior engineer at StartupXYZ 
    from 2018 to 2021. Sarah is skilled in Python, distributed systems, 
    and machine learning. She lives at 123 Market St, San Francisco, CA 94102.
    """

    result = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Extract structured information from this bio",
            "context": {"bio": bio},
            "imodel": model.name,
            "response_model": Person,
            "return_as": "model",
        },
    )

    print(f"Name: {result.name}")
    print(f"Age: {result.age}")
    print(f"Occupation: {result.occupation}")
    print("\nAddress:")
    if result.address:
        print(f"  {result.address.street}")
        print(f"  {result.address.city}, {result.address.postal_code}")
        print(f"  {result.address.country}")
    print("\nEmployment History:")
    for job in result.employment_history:
        current = " (current)" if job.is_current else ""
        end = job.end_year or "present"
        print(f"  - {job.position} at {job.company} ({job.start_year}-{end}){current}")
    print(f"\nSkills: {', '.join(result.skills)}")

    return result


result = await extract_person_info()

## 4. Validation and Retries

lionpride can automatically retry when the model produces invalid output.

In [ ]:
class StrictScore(BaseModel):
    """Score with strict validation."""

    score: int = Field(..., ge=1, le=10, description="Score from 1 to 10")
    justification: str = Field(
        ..., min_length=20, description="At least 20 character justification"
    )


async def validated_scoring():
    """Get validated scores with retry on failure."""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0)
    session.services.register(model)
    branch = session.create_branch(name="scoring")

    result = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Rate this code quality from 1-10",
            "context": {"code": "def add(a,b): return a+b"},
            "imodel": model.name,
            "response_model": StrictScore,
            "return_as": "model",
            "max_retries": 2,  # Retry up to 2 times on validation failure
            "strict_validation": False,  # Don't raise on failure
        },
    )

    if isinstance(result, dict) and result.get("validation_failed"):
        print(f"Validation failed: {result['error']}")
        print(f"Raw response: {result['raw'][:200]}...")
    else:
        print(f"Score: {result.score}/10")
        print(f"Justification: {result.justification}")

    return result


result = await validated_scoring()

## 5. Using operate() with Reasoning

The `operate()` function adds optional reasoning fields to structured outputs.

In [ ]:
class Decision(BaseModel):
    """A decision with options analysis."""

    choice: str = Field(..., description="The recommended choice")
    alternatives: list[str] = Field(default_factory=list, description="Other options considered")
    risk_level: Literal["low", "medium", "high"] = Field(..., description="Risk assessment")


async def decision_with_reasoning():
    """Get structured decision with explicit reasoning."""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o", temperature=0)
    session.services.register(model)
    branch = session.create_branch(name="decision")

    result = await operate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Recommend a database for a high-traffic e-commerce site",
            "context": {
                "requirements": [
                    "10M daily active users",
                    "Complex product catalog",
                    "Real-time inventory",
                    "Strong consistency required",
                ]
            },
            "imodel": model.name,
            "response_model": Decision,
            "reason": True,  # Add reasoning field
        },
    )

    print("=== Reasoning ===")
    if hasattr(result, "reason") and result.reason:
        print(f"Title: {result.reason.title}")
        print(f"Confidence: {result.reason.confidence}")
        print(f"Content: {result.reason.content[:200]}...")

    print("\n=== Decision ===")
    if hasattr(result, "decision"):
        print(f"Choice: {result.decision.choice}")
        print(f"Risk: {result.decision.risk_level}")
        print(f"Alternatives: {result.decision.alternatives}")

    return result


result = await decision_with_reasoning()

## 6. Enums and Literal Types

Use `Literal` or `Enum` for constrained choices.

In [ ]:
from enum import Enum


class Priority(str, Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"


class TaskClassification(BaseModel):
    """Task classification result."""

    task: str = Field(..., description="The task description")
    category: Literal["bug", "feature", "refactor", "docs", "test"] = Field(
        ..., description="Task category"
    )
    priority: Priority = Field(..., description="Priority level")
    estimated_hours: float = Field(..., ge=0.5, le=40, description="Estimated hours")
    tags: list[str] = Field(default_factory=list, description="Relevant tags")


async def classify_task():
    """Classify a task with constrained options."""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0)
    session.services.register(model)
    branch = session.create_branch(name="classify")

    result = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Classify this task",
            "context": {
                "task_description": "The login page crashes when users enter "
                "special characters in the password field. "
                "This affects production users."
            },
            "imodel": model.name,
            "response_model": TaskClassification,
            "return_as": "model",
        },
    )

    print(f"Task: {result.task}")
    print(f"Category: {result.category}")
    print(f"Priority: {result.priority.value}")
    print(f"Estimated: {result.estimated_hours} hours")
    print(f"Tags: {result.tags}")

    return result


result = await classify_task()

## 7. Lists and Multiple Items

Extract multiple structured items from content.

In [ ]:
class Entity(BaseModel):
    """Named entity."""

    name: str
    entity_type: Literal["person", "organization", "location", "date", "money"]
    context: str = Field(..., description="How the entity appears in text")


class EntityExtractionResult(BaseModel):
    """Named entity extraction result."""

    entities: list[Entity] = Field(default_factory=list)
    total_count: int = Field(..., description="Total entities found")


async def extract_entities():
    """Extract multiple entities from text."""
    session = Session()
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0)
    session.services.register(model)
    branch = session.create_branch(name="ner")

    text = """
    Apple Inc. announced yesterday that CEO Tim Cook will visit 
    their new $5 billion campus in Austin, Texas on March 15, 2025.
    The meeting with Governor Abbott is expected to discuss the 
    company's $1 billion investment in local manufacturing.
    """

    result = await communicate(
        session=session,
        branch=branch,
        parameters={
            "instruction": "Extract all named entities from this text",
            "context": {"text": text},
            "imodel": model.name,
            "response_model": EntityExtractionResult,
            "return_as": "model",
        },
    )

    print(f"Found {result.total_count} entities:\n")
    for entity in result.entities:
        print(f"  [{entity.entity_type}] {entity.name}")
        print(f"    Context: {entity.context}")

    return result


result = await extract_entities()

## Summary

### Key Concepts

| Parameter | Description |
|-----------|-------------|
| `response_model` | Pydantic model for JSON Schema validation |
| `return_as="model"` | Return validated Pydantic instance |
| `max_retries` | Retry count on validation failure |
| `strict_validation` | Raise exception on failure (default: False) |

### Best Practices

- Use `Field(description=...)` for better LLM understanding
- Use `Literal` for constrained string choices
- Use validators (`ge`, `le`, `min_length`) for data quality
- Add `default_factory=list` for optional lists
- Use `reason=True` with `operate()` for chain-of-thought

### Schema Efficiency

lionpride uses TypeScript notation in prompts, typically achieving **60-80% token reduction** compared to raw JSON Schema. This happens automatically when you use `response_model`.

### Next Steps

- See notebook 03_lndl.ipynb for LNDL cognitive programming
- See notebook 04_tool_calling.ipynb for ReAct with tools
- See notebook 05_multi_turn.ipynb for conversation context